# Ginelli Checkpoint Development

Notebook for editing Ginelli code to checkpoint.

In [270]:
# Standard Imports

import numpy as np
import time as tm
import pickle
from tqdm.notebook import tqdm
import xarray as xr
import sys
import os

## Working notes:

- First finish checkpoint decorator, this takes a step of the ginelli algorithm and makes it in to a loop and saving function

- Need a self.stage attribute, checkpoint decorator should update this. 

- Need and self.stage_IO_directory attribute. should correspond to self.stage

In [240]:
class TangentIntegratorL96:
    
    """Integrates the L96 ODEs and it's tangent dynamics simultaneously."""
    def __init__(self, K=36, J=10, h=1, Ff=6, Fs=10, c=10, dt=0.001, save_every=10,
                 X_init=None, Y_init=None, dx_init=None, dy_init=None, noprog=False,):
        """param, save_every: steps at which we save output."""
        
        # Model parameters
        self.K, self.J, self.h, self.Ff, self.Fs, self.c, self.dt = K, J, h, Ff, Fs, c, dt
        self.b = np.sqrt(J * c) # b restriction
        self.size = self.K + (self.J * self.K) # Number of variables
        self.save_every = save_every

        # Step counts
        self.step_count = 0 # Number of integration steps

        # Progress Bars
        self.noprog = noprog

        # Non-linear Variables
        self.X = np.random.rand(self.K) if X_init is None else X_init.copy() # Random IC if none given
        self.Y = np.zeros(self.K * self.J) if Y_init is None else Y_init.copy() * self.b # ALL the y's
        self._history_X = [self.X.copy()]
        self._history_Y = [self.Y.copy()] # Y's in a list, remember they are in natural lists of length K

        # TLE Variables
        self.dx = np.random.rand(self.K) if dx_init is None else dx_init.copy()
        self.dy = np.random.rand(self.K * self.J) if dy_init is None else dy_init.copy()
        
        self._history_dx = [self.dx.copy()]
        self._history_dy = [self.dy.copy()] # Y's in a list, remember they are in natural lists of length K
    
    def _rhs_X_dt(self, X, Y):
        """Compute the right hand side of the X-ODE. Note this has been scaled."""

        dXdt = (
                np.roll(X, 1) * (np.roll(X, -1) - np.roll(X, 2)) -
                X + self.Fs - self.h * Y.reshape(self.K, self.J).mean(1) # Using Y mean
        )
        return self.dt * dXdt

    def _rhs_Y_dt(self, X, Y):
        """Compute the right hand side of the Y-ODE."""
        dYdt = self.c * (
                          np.roll(Y, -1) * ( np.roll(Y, 1) - np.roll(Y, -2) )
                       - Y + self.Ff + self.h * np.repeat(X, self.J) # repeat so x's match y's
               )
        return self.dt * dYdt

    def _rhs_dx_dt(self, X, dx, dy):
        """Computhe rhs of the dx-ODE"""
        ddxdt = (
                    np.roll(dx, 1) * ( np.roll(X, -1) - np.roll(X, 2) )
                    + np.roll(X, 1) * ( np.roll(dx, -1) - np.roll(dx, 2) ) - dx
                   - self.h * dy.reshape(self.K, self.J).mean(1)
        )
        return self.dt * ddxdt

    def _rhs_dy_dt(self, Y, dx, dy):
        """Computhe rhs of the dy-ODE"""
        ddydt = self.c * (
                            np.roll(dy, -1) * (np.roll(Y, 1) - np.roll(Y, -2) )
                          + np.roll(Y, -1) * (np.roll(dy, 1) - np.roll(dy, -2) )
                        - dy + self.h * np.repeat(dx, self.J)
        )
        return self.dt * ddydt

    def _rhs_dt(self, X, Y, dx, dy):
        return self._rhs_X_dt(X, Y), self._rhs_Y_dt(X, Y), self._rhs_dx_dt(X, dx, dy), self._rhs_dy_dt(Y, dx, dy)

    def _step(self):
        """Integrate one time step"""

        # RK Coefficients
        k1_X, k1_Y, k1_dx, k1_dy = self._rhs_dt(self.X, self.Y,
                                                self.dx, self.dy)
        k2_X, k2_Y, k2_dx, k2_dy = self._rhs_dt(self.X + k1_X / 2, self.Y + k1_Y / 2,
                                                self.X + k1_X / 2, self.dy + k1_dy / 2)
        k3_X, k3_Y, k3_dx, k3_dy = self._rhs_dt(self.X + k2_X / 2, self.Y + k2_Y / 2,
                                               self.dx + k2_dx / 2, self.dy + k2_dy / 2)
        k4_X, k4_Y, k4_dx, k4_dy = self._rhs_dt(self.X + k3_X, self.Y + k3_Y,
                                               self.dx + k3_dx / 2, self.dy + k3_dy / 2)

        # Update State
        self.X += 1 / 6 * (k1_X + 2 * k2_X + 2 * k3_X + k4_X)
        self.Y += 1 / 6 * (k1_Y + 2 * k2_Y + 2 * k3_Y + k4_Y)
        self.dx += 1 / 6 * (k1_dx + 2 * k2_dx + 2 * k3_dx + k4_dx)
        self.dy += 1 / 6 * (k1_dy + 2 * k2_dy + 2 * k3_dy + k4_dy)
        self.step_count += 1
        
        # Store history
        if (self.step_count%self.save_every == 0):
            self._history_X.append(self.X.copy())
            self._history_Y.append(self.Y.copy())
            self._history_dx.append(self.dx.copy())
            self._history_dy.append(self.dy.copy())

    def integrate(self, time):  
        """time: how long we integrate for in adimensional time."""
        steps = int(time / self.dt)
        for n in tqdm(range(steps), disable = self.noprog):
            self._step()
            
    def clear_history(self):
        """clears history of variable evolution"""
        self._history_X = []
        self._history_Y = []
        self._history_dx = []
        self._history_dy = []
    
    def set_state(self, x, tangent_x):
        """x is [X, Y]. tangent_x is [dx, dy]"""
        self.X = x[:self.K]
        self.Y = x[self.K:]
        self.dx = tangent_x[: self.K]
        self.dy = tangent_x[self.K: ] 
    
    @property
    def state(self):
        """Where we are"""
        return np.concatenate([self.X, self.Y, self.dx, self.dy])
    
    @property
    def time(self):
        """a-dimensional time"""
        return self.dt * self.step_count
    
    @property
    def parameter_dict(self):
        param = {
        'h': self.h, # L96
        'Fs': self.Fs,
        'Ff': self.Ff,
        'c': self.c,
        'J': self.J,
        'K': self.K,
        'Number of variables': self.size,
        'b': self.b,
        'dt': self.dt
        }
        return param
    
    @property
    def run_data(self):
        """Returns xarray of run data"""
        if (len(self._history_X) == 0):
            print('No history to look at!')
        dic = {}
        _time = np.arange(self.step_count - (len(self._history_X) - 1) * self.save_every,
                          self.step_count + 1, self.save_every) * self.dt
        # Notice we add step count. Might change run_data if you have cleared history.
        
        dic['X'] = xr.DataArray(self._history_X, dims=['time', 'x'], name='X',
                                coords = {'time': _time,'x': np.arange(self.K)})
        dic['Y'] = xr.DataArray(self._history_Y, dims=['time', 'y'], name='Y',
                                coords = {'time': _time,'y': np.arange(self.K * self.J)})

        dic['dx'] = xr.DataArray(self._history_dx, dims=['time', 'x'], name='dx',
                                coords = {'time': _time,'x': np.arange(self.K)})
        dic['dy'] = xr.DataArray(self._history_dy, dims=['time', 'y'], name='dy',
                                coords = {'time': _time,'y': np.arange(self.K * self.J)})
        # Slow Variables above fast ones
        dic['X_repeat'] = xr.DataArray(np.repeat(self._history_X, self.J, axis=1),
                                   dims=['time', 'y'], name='X_repeat',
                                    coords = {'time': _time,'y': np.arange(self.K * self.J)})# X's above the y's
        
        return xr.Dataset(dic, attrs = self.parameter_dict)

In [241]:
runner = TangentIntegratorL96(K = 2, J = 2)

In [242]:
runner.integrate(1)

In [243]:
runner.run_data

<xarray.Dataset>
Dimensions:   (time: 101, x: 2, y: 4)
Coordinates:
  * time      (time) float64 0.0 0.01 0.02 0.03 0.04 ... 0.96 0.97 0.98 0.99 1.0
  * x         (x) int64 0 1
  * y         (y) int64 0 1 2 3
Data variables:
    X         (time, x) float64 0.7154 0.3722 0.8031 ... 4.733 4.823 4.718
    Y         (time, y) float64 0.0 0.0 0.0 0.0 ... 3.72 -1.725 8.83 6.428
    dx        (time, x) float64 0.5908 0.3119 0.5747 ... -0.3926 -0.7765 -0.3474
    dy        (time, y) float64 0.6063 0.8455 0.8156 ... 10.6 26.07 -31.48
    X_repeat  (time, y) float64 0.7154 0.7154 0.3722 ... 4.823 4.718 4.718
Attributes:
    h:                    1
    Fs:                   10
    Ff:                   6
    c:                    10
    J:                    2
    K:                    2
    Number of variables:  6
    b:                    4.47213595499958
    dt:                   0.001

In [244]:
runner.clear_history()

In [245]:
runner.integrate(0.2)

In [246]:
runner.run_data

<xarray.Dataset>
Dimensions:   (time: 20, x: 2, y: 4)
Coordinates:
  * time      (time) float64 1.01 1.02 1.03 1.04 1.05 ... 1.17 1.18 1.19 1.2
  * x         (x) int64 0 1
  * y         (y) int64 0 1 2 3
Data variables:
    X         (time, x) float64 4.861 4.702 4.905 4.705 ... 5.003 5.239 5.019
    Y         (time, y) float64 3.82 -3.182 3.733 10.24 ... 2.082 -2.156 11.19
    dx        (time, x) float64 -0.839 -0.4308 -0.9046 ... -0.9458 -0.8988
    dy        (time, y) float64 11.18 8.506 49.93 -18.7 ... -14.84 -6.592 6.834
    X_repeat  (time, y) float64 4.861 4.861 4.702 4.702 ... 5.239 5.019 5.019
Attributes:
    h:                    1
    Fs:                   10
    Ff:                   6
    c:                    10
    J:                    2
    K:                    2
    Number of variables:  6
    b:                    4.47213595499958
    dt:                   0.001

In [276]:
def posQR(M):
    """ Returns QR decomposition of a matrix with positive diagonals on R.
    Parameter, M: Array that is being decomposed
    """
    Q, R = np.linalg.qr(M) # Performing QR decomposition
    signs = np.diag(np.sign(np.diagonal(R))) # Matrix with signs of R diagonal on the diagonal
    Q, R = np.dot(Q, signs), np.dot(signs, R) # Ensuring R Diagonal is positive
    return Q, R

class GinelliL96(TangentIntegratorL96):
    """Builds upon a tangent integrator class to perform the Ginelli algorithm"""
    
    def __init__(self, ka, kb, kc, tau=0.1, K=36, J=10, h=1, Ff=6, Fs=10, c=10, dt=0.001, save_every=10,
                 X_init=None, Y_init=None, dx_init=None, dy_init=None, noprog=True):
        
        self.save_every = save_every # how many tau steps we save
        run_save_step = (self.save_every * tau)/dt
        
        super().__init__(K=K, J=J, h=h, Ff=Ff, Fs=Fs, c=c, dt=dt, save_every=run_save_step,
                 X_init=X_init, Y_init=Y_init, dx_init=dx_init, dy_init=dy_init, noprog=noprog)
        
        # Ginelli Parameters
        self.ka, self.kb, self.kc, self.tau = ka, kb, kc, tau
    
        # Ginelli Variables
        self.P = np.random.rand(self.size, self.size) # Stretched Matrix
        eps = 0.001
        self.oldQ = eps * np.identity(self.size)
        self.oldQ[0, 1] = eps * 1
        self.R = np.random.rand(self.size, self.size)  # Stretching rates
        self._history_R = []
        self.oldA = np.identity(self.size) # Initial A
        self.oldA[0, 1] = 1

        # Lyapunov Spectra
        self.FTBLE = np.random.rand(int(self.kb), self.size)
        self.FTCLE = np.random.rand(int(self.kb), self.size)
        self._history_FTBLE = [] # For storing time series
        self._history_FTCLE = []

        # Lyapunov Vectors
        self.BLV = np.random.rand(self.size, self.size)
        self.CLV = np.random.rand(self.size, self.size)
        self._history_BLV = []
        self._history_CLV = []
        
        self.ginelli_count = 0 # Number of QR steps performed
        
    def _ginelli_step(self):
        """One QR step. Take old Q, stretch it, do a QR decomposition."""

        # Where we are in phase space before ginelli step
        phase_state = self.state[:self.size]

        # Stretching first column
        self.set_state(phase_state, self.oldQ.T[0]) # First column of Q is ic for TLE
        self.integrate(self.tau)

        # Saving Output
        self.P[:, 0] = np.append(self.dx, self.dy)

        # Stretching the rest of the columns
        for i, column in enumerate(self.oldQ.T[1:]): # Evolve each CLV
            self.set_state(phase_state, column)
            self._integrate(self.tau)
            self.P[:, i] = np.append(self.dx, self.dy) # Building P

        # QR decomposition
        self.oldQ, self.R = posQR(self.P)
        self.ginelli_count += 1

    
    @property
    def le_data(self):
        """Returns xarray of LEs and LVs"""
        if (len(self._history_X) == 0):
            print('No history to look at!')
        dic = {}
        _time = np.arange(self.ginelli_count - (len(self._history_FTBLE) - 1) * self.save_every,
                          self.ginelli_count + 1, self.save_every) * self.tau 
        # Notice we add step count. Might change run_data if you have cleared history.
        
        dic['FTBLE'] = xr.DataArray(self._history_FTBLE, dims=['time', 'le_index'], name='FTBLE',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size)})
        dic['FTCLE'] = xr.DataArray(self._history_FTCLE, dims=['time', 'le_index'], name='FTCLE',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size)})
        dic['BLV'] = xr.DataArray(self._history_BLV, dims=['time', 'component', 'le_index'], name='BLV',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size),
                                  'component': np.arange(self.size), 'le_index':np.arange(1, 1 + self.size)})
        dic['CLV'] = xr.DataArray(self._history_CLV, dims=['time', 'component', 'le_index'], name='CLV',
                                coords = {'time': _time,'le_index':np.arange(1, 1 + self.size),
                                  'component': np.arange(self.size), 'le_index':np.arange(1, 1 + self.size)})
        
        return xr.Dataset(dic, attrs = self.ginelli_dict)
        

In [277]:
ginelliRunner = GinelliL96(1,1,1,1)

In [278]:
ginelliRunner.integrate(1)

In [279]:
ginelliRunner.le_data

ValueError: different number of dimensions on data and dims: 1 vs 2

In [280]:
ginelliRunner.run_data

<xarray.Dataset>
Dimensions:   (time: 1, x: 36, y: 360)
Coordinates:
  * time      (time) float64 1.0
  * x         (x) int64 0 1 2 3 4 5 6 7 8 9 10 ... 26 27 28 29 30 31 32 33 34 35
  * y         (y) int64 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
Data variables:
    X         (time, x) float64 0.2875 0.1375 0.448 ... 0.4724 0.4779 0.9347
    Y         (time, y) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    dx        (time, x) float64 0.07541 0.3035 0.9686 ... 0.9129 0.6848 0.0756
    dy        (time, y) float64 0.1909 0.3639 0.596 ... 0.3416 0.5254 0.4112
    X_repeat  (time, y) float64 0.2875 0.2875 0.2875 ... 0.9347 0.9347 0.9347
Attributes:
    h:                    1
    Fs:                   10
    Ff:                   6
    c:                    10
    J:                    10
    K:                    36
    Number of variables:  396
    b:                    10.0
    dt:                   0.001